# IMPORTING LIBRARIES

In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers, applications
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# IMPORTING THE RESNET50 AND USING TRANSFER LEARNING FOR CUSTOM MODEL (CAT OR DOG CLASSIFICATION)

In [13]:
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(256,256,3))

for layer in base_model.layers:
    layer.trainable = False

In [14]:
x = layers.Flatten()(base_model.output)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(1, activation='sigmoid')(x)

In [15]:
model = models.Model(inputs=base_model.input, outputs=x)

# USING IMAGEDATAGENEARTOR FOR IMAGE OPTIMIZATION

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## TRAINING SET

In [16]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip = True,
    shear_range = 0.2,
    zoom_range = 0.2
)

train_dataset = train_datagen.flow_from_directory(
    'dataset/training_set',
    shuffle=True,
    batch_size = 32,
    class_mode = 'binary'
)

Found 8000 images belonging to 2 classes.


## TEST SET

In [17]:
test_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    shear_range = 0.2,
    zoom_range = 0.2
)

test_dataset = test_datagen.flow_from_directory(
    'dataset/test_set',
    shuffle=True,
    batch_size = 32,
    class_mode = 'binary'
)

Found 2000 images belonging to 2 classes.


# COMPILING AND TRAINING THE CUSTOM MODEL

In [18]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
model.fit(train_dataset, epochs=10, validation_data=test_dataset, batch_size=32)

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1205s 5s/step - accuracy: 0.7637 - loss: 0.6630 - val_accuracy: 0.9060 - val_loss: 0.2291
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1222s 5s/step - accuracy: 0.9169 - loss: 0.2015 - val_accuracy: 0.9215 - val_loss: 0.1921
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1219s 5s/step - accuracy: 0.9322 - loss: 0.1649 - val_accuracy: 0.9210 - val_loss: 0.1879
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1230s 5s/step - accuracy: 0.9439 - loss: 0.1482 - val_accuracy: 0.9310 - val_loss: 0.1745
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1217s 5s/step - accuracy: 0.9501 - loss: 0.1259 - val_accuracy: 0.9085 - val_loss: 0.2291
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1219s 5s/step - accuracy: 0.9549 - loss: 0.1138 - val_accuracy: 0.9130 - val_loss: 0.2147
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1223s 5s/step - accuracy: 0.9528 - loss: 0.1176 - val_accuracy: 0.9130 - val_loss: 0.2101
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1244s 5s/step - accuracy: 0.9585 - loss: 0.1034 - 

# SAVING THE MODEL INTO H5 AND KERAS

In [20]:
model.save('cat_dog_classification_model.h5')

In [22]:
model.save('cat_dog_classification_model.keras')

# TESTING ON AN IMAGE

In [23]:
from tensorflow.keras.preprocessing import image

In [27]:
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size=(256,256))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
train_dataset.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 608ms/step
cat


# MODEL SUMMARY

In [28]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 256, 256, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 256, 256, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 128, 128, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 64, 64, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 64, 64, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 64, 64, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 32, 32, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 16, 16, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     4,194,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,298,373 (104.14 MB)

 Trainable params: 4,194,561 (16.00 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

 Optimizer params: 8,389,124 (32.00 MB)